In [67]:
from __future__ import division
from PIL import Image
import imagehash
import argparse
import shelve
import glob
import os
import re
import random
import time
import binascii
import hashlib
from bisect import bisect_right
from heapq import heappop, heappush


In [77]:
# Time this step.
# print ('\nGenerating random hash functions...')
# Record the maximum shingle ID that we assigned.
maxShingleID = 2**32-1

# We need the next largest prime number above 'maxShingleID'.
# I looked this value up here: 
# http://compoasso.free.fr/primelistweb/page/prime/liste_online_en.php
nextPrime = 4294967311


Generating random hash functions...


In [78]:
def pickRandomCoeffs(k):
    # Create a list of 'k' random values.
    randList = []

    while k > 0:
    # Get a random shingle ID.
        randIndex = random.randint(0, maxShingleID) 

        # Ensure that each random number is unique.
        while randIndex in randList:
            randIndex = random.randint(0, maxShingleID) 

        # Add the random number to the list.
        randList.append(randIndex)
        k = k - 1
    return randList

In [79]:
print(pickRandomCoeffs(5))

[3057765316, 1989347376, 621872310, 2866481810, 2098430060]


In [164]:
coeffA = pickRandomCoeffs(10)
coeffB = pickRandomCoeffs(10)

# List of documents represented as signature vectors
signatures = []

# Rather than generating a random permutation of all possible shingles, 
# we'll just hash the IDs of the shingles that are *actually in the document*,
# then take the lowest resulting hash code value. This corresponds to the index 
# of the first shingle that you would have encountered in the random order.

# For each document...
image_hashset = []
# for imagePath in glob.glob("dataset1" + "/*.jpg"):
for imagePath in glob.glob("101_ObjectCategories/accordion" + "/*.jpg"):
    
    # Get the hashfunction for the image
    image = Image.open(imagePath)
    h = str(imagehash.dhash(image))
    image_hashset.append(h)
  

image_hashset2 = []
# for imagePath in glob.glob("dataset2" + "/*.jpg"):
for imagePath in glob.glob("101_ObjectCategories/accordion2" + "/*.jpg"):

    # Get the hashfunction for the image
    image = Image.open(imagePath)
    h = str(imagehash.dhash(image))
    image_hashset2.append(h)


# image_hashset_test = []
# image_test = Image.open("image_0020.jpg")
# h_test = str(imagehash.dhash(image_test))
# image_hashset_test.append(h_test)
print(image_hashset[0])
print('==================')
print(image_hashset[5])
print(len(image_hashset[5]))

64c6cee6331382c1
64c28f0723339ecd
16


In [161]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [162]:
similar(image_hashset[0],image_hashset[5])

0.4375

In [153]:
signature1 = []
signature2 = []
hh = []

# For each of the random hash functions...
for i in range(0, 10):
    # For each of the shingles actually in the document, calculate its hash code
    # using hash function 'i'. 

    # Track the lowest hash ID seen. Initialize 'minHashCode' to be greater than
    # the maximum possible value output by the hash.
    minHashCode1 = nextPrime + 1
    minHashCode2 = nextPrime + 1

    # For each shingle in the document...
    for image_hash in image_hashset:
        # Evaluate the hash function.
        image_ID = abs(hash(image_hash)) % (10 ** 8)
        hashCode = (coeffA[i] * image_ID + coeffB[i]) % nextPrime 
        hh.append(hashCode)
        # Track the lowest hash code seen.
        if hashCode < minHashCode1:
            minHashCode1 = hashCode

    # Add the smallest hash code value as component number 'i' of the signature.
    signature1.append(minHashCode1)
    
#     image_ID_test = abs(hash(h_test)) % (10 ** 8)
#     hashCode_test = (coeffA[i] * image_ID + coeffB[i]) % nextPrime 

#     # Track the lowest hash code seen.
#     if hashCode_test < minHashCode2:
#         minHashCode2 = hashCode_test
#     print(minHashCode2)
    
    for image_hash in image_hashset2:
        # Evaluate the hash function.
        image_ID = abs(hash(image_hash)) % (10 ** 8)
        hashCode = (coeffA[i] * image_ID + coeffB[i]) % nextPrime 

        # Track the lowest hash code seen.
        if hashCode < minHashCode2:
            minHashCode2 = hashCode

    # Add the smallest hash code value as component number 'i' of the signature.
    signature2.append(minHashCode2)

print(signature1)
print(signature2)


[95041122, 77781479, 61800884, 34315693, 132215246, 5540555, 17261061, 46891330, 96953454, 7747837]
[77960296, 77781479, 61800884, 34315693, 132215246, 5540555, 17261061, 46891330, 96953454, 7747837]


In [154]:
same_items = 0
for i in range(0, 10):
    if (signature1[i]==signature2[i]):
        same_items += 1
print ("\nEstimated Jaccard Similarity = " + str(same_items/10))
# same = 0
# for i in range(len(hh)):
#     for j in range(len(signature2)):
#         if (hh[i] == signature2[j] ):
#             same += 1
# print(same)


Estimated Jaccard Similarity = 0.9


In [155]:
exact_num = len(set(image_hashset).intersection(image_hashset2))
exact_total = len(set(image_hashset).union(image_hashset2))

print("\nNumber of common images: "+ str(exact_num))
print("Number of total images: " + str(exact_total))
J = exact_num/exact_total
print("\nExact Jaccard Similarity = " + str(J))



Number of common images: 45
Number of total images: 65

Exact Jaccard Similarity = 0.6923076923076923


In [115]:
from datasketch import MinHash, MinHashLSH

set1 = set(['minhash', 'is', 'a', 'probabilistic', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'datasets'])
set2 = set(['minhash', 'is', 'a', 'probability', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'documents'])
set3 = set(['minhash', 'is', 'probability', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'documents'])

m1 = MinHash(num_perm=128)
m2 = MinHash(num_perm=128)
m3 = MinHash(num_perm=128)
for d in set1:
    m1.update(d.encode('utf8'))
for d in set2:
    m2.update(d.encode('utf8'))
for d in set3:
    m3.update(d.encode('utf8'))

# Create LSH index
lsh = MinHashLSH(threshold=0.5, num_perm=128)
lsh.insert("m2", m2)
lsh.insert("m3", m3)
result = lsh.query(m1)
print("Approximate neighbours with Jaccard similarity > 0.5", result)

Approximate neighbours with Jaccard similarity > 0.5 ['m3', 'm2']
